In [ ]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

from pathlib import Path
import zipfile

folder_name = "terraclimate_aet_data"

source_dir = Path(f"./{folder_name}/")      # folder containing 60 CSVs
zip_path   = Path(f"/tmp/{folder_name}.zip")

csv_files = sorted(source_dir.glob("*.csv"))  # or "**/*.csv" for recursive

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {source_dir}")

with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
    for file in csv_files:
        zf.write(file, arcname=file.name)

print(f"Created: {zip_path} ({len(csv_files)} files)")

session.sql(f"""
    PUT file:///tmp/{folder_name}.zip
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")